In [1]:
from pandas.io.parsers import read_csv
import numpy as np

np.random.seed(0) # REMOVE FOR FINAL SOLUTION


In [2]:
#Data Splitted.
def splitData(data):
    shuffled = data.copy()
    np.random.shuffle(shuffled)

    testCount = int(shuffled.shape[0]*1/3)
    trainCount = shuffled.shape[0]-testCount

    trainSet = shuffled[0:trainCount]
    testSet = shuffled[trainCount:]

    return trainSet, testSet

def separateInputOutput(dataset):
    outputData = dataset[:,3] # Output
    inputData = np.concatenate((dataset[:,0:3],dataset[:,4:7],dataset[:,8:10]),axis=1) # Inputs
    return inputData, outputData

#ML ALG
def initializeWeights(x,y):
    N = x.shape[0]
    
def normaliseData(dataArray, normType = "range"):
    normalisedDataList = []

    if normType == "range":
        maxVal = dataArray.max()
        minVal = dataArray.min()
        for val in dataArray:
            normalisedDataList.append((val-minVal)/(maxVal-minVal))
        return np.array(normalisedDataList)

    elif normType == "z":
        mean = dataArray.mean()
        stdDev =  getStandardDeviation(dataArray)
        for val in dataArray:
            normalisedDataList.append((val-mean)/stdDev)
        return np.array(normalisedDataList)

def getStandardDeviation(dataArray):
    n = dataArray.shape[0]
    mean = dataArray.mean()
    sumOfSquareDiffs = 0
    for val in dataArray:
        sumOfSquareDiffs += (val - mean)**2

    variance = sumOfSquareDiffs/(n-1)
    stdDev = variance**(1/2)
    return stdDev


In [3]:
def getUniqueClassCount(listToCount):
    classes = []
    for item in listToCount:
        if item not in classes:
            classes.append(item)

    counts = []
    for itemClass in classes:
        counts.append(listToCount.count(itemClass))
    return counts

def getGiniLeaf(counts):
    total = np.sum(counts)
    giniLeaf = 1
    for count in counts:
        giniLeaf -= (count/total)**2
    return giniLeaf

def getGiniNode(sortedList, threshold):
        lessThan = []
        greaterThan = []

        for i in range(len(sortedList)):
            x = sortedList[i][0]
            # print(x, threshold)
            if x < threshold:
                lessThan.append(sortedList[i][1])
            else:
                greaterThan.append(sortedList[i][1])

        countsLT = getUniqueClassCount(lessThan)
        countsGT = getUniqueClassCount(greaterThan)
        giniLT = getGiniLeaf(countsLT)
        giniGT = getGiniLeaf(countsGT)
        # print(giniLT,giniGT)
        # print(np.sum(countsLT),np.sum(countsGT))
        totalCount = len(sortedList)
        # print(totalCount)
        giniNode = ((np.sum(countsLT)/totalCount)*giniLT) + ((np.sum(countsGT)/totalCount)*giniGT)
        # print(giniNode)
        return giniNode


In [4]:
def getSplitValue(trainIn, trainOut, attributeIndex):
    attribute = trainIn[:, attributeIndex].copy() # Output
    combined = []
    for j,sample in enumerate(attribute):
        combined.append((sample,trainOut[j]))
    attribute.sort()
    sortedList = [tuple for x in attribute for tuple in combined if tuple[0] == x]


    thresholds = []
    for j in range(len(attribute)-1):
        thresholds.append((attribute[j]+attribute[j+1])/2)

    giniNodes = []
    for threshold in thresholds:
        giniNodes.append(getGiniNode(sortedList, threshold))
    # print(giniNodes)
    minNodeGini = min(giniNodes)
    minGiniIndex = giniNodes.index(minNodeGini)
    splitVal = thresholds[minGiniIndex]
    return splitVal, minNodeGini
   

In [5]:
def train(trainIn,trainOut):
    ginis = []
    splitVals = []
    for i in range(trainIn.shape[1]):
        splitVal, minGini = getSplitValue(trainIn,trainOut, i)
        ginis.append(minGini)
        splitVals.append(splitVal)
        # print(splitVal, minGini)
    # print("Root is input data attribute", ginis.index(min(ginis)))
    branchData(testIn,testOut,ginis.index(min(ginis)),splitVals[ginis.index(min(ginis))]) #

In [6]:
def branchData(X, y, splitAttributeIndex, splitValue):
    lessThan_X = []
    lessThan_y = []
    greaterThan_X = []
    greaterThan_y = []
    
    for i in range(len(X)):
        if X[i,splitAttributeIndex] < splitValue:
            lessThan_X.append(X[i,:])
            lessThan_y.append(y[i])
        else:
            greaterThan_X.append(X[i,:])
            greaterThan_y.append(y[i])
    print(lessThan_X)

        

In [7]:
#Reading in data
dataframe = read_csv("beer.txt",delim_whitespace=True,header=None)

# Preparing Data
dataset = dataframe.values
trainSet, testSet = splitData(dataset)

testIn, testOut = separateInputOutput(testSet)
trainIn, trainOut = separateInputOutput(trainSet)

N =trainIn.shape[0]
Wi = 1/N

#running ml alg.
train(trainIn,trainOut) #ML ALG RAN HERE 


[array([38.09292035, 0.261521856, 1.7318181819999998, 3.618461538, 17.68,
       7.556157895, 14.76, 81.21142857], dtype=object), array([43.93362832, 0.254582118, 2.451818182, 3.8492307689999996, 16.24,
       5.2190526319999995, 9.0, 62.60714286], dtype=object), array([38.53539823, 0.286716054, 1.259090909, 3.7938461539999997, 16.1,
       4.326368421000001, 15.0, 62.56], dtype=object), array([38.44690265, 0.230198978, 2.308181818, 3.907692308, 18.55,
       8.877894737, 9.36, 65.43428571], dtype=object), array([42.561946899999995, 0.238442508, 0.774545455, 3.526153846, 18.74,
       8.155894737, 9.0, 51.59714286], dtype=object), array([45.52654867, 0.189464469, 1.750909091, 3.8892307689999996, 16.36,
       6.581526316000001, 11.76, 54.91142857], dtype=object), array([39.90707965, 0.24168399899999998, 1.155454545, 3.584615385, 16.67,
       6.359421052999999, 16.32, 73.16857143], dtype=object), array([45.0840708, 0.39002710399999996, 0.847272727, 3.68, 16.09,
       11.42347368, 11.8